# Values Tutorial

For functions that have just a few inputs and outputs, the function codegen approach works best. For growing to more complex use cases, we need the ability to build up large collections of symbolic expressions with a structured input/output specification, which will be used for code generation.

This specification is done by building expressions into [Values](../api/symforce.values.values.html#symforce.values.values.Values) objects, which are ordered dict-like containers:

In [ ]:
# Setup
import symforce
symforce.set_backend('sympy')
symforce.set_log_level('warning')

from symforce import geo
from symforce import sympy as sm
from symforce.notebook_util import display, display_code, display_code_file

In [ ]:
from symforce.values import Values

inputs = Values(
    x=sm.Symbol('x'),
    y=geo.Rot2.symbolic('c')
)
display(inputs)

The `.add()` method can add a symbol using its name as the key:

In [ ]:
inputs.add(sm.Symbol('foo'))
display(inputs)

Adding sub-values are well encouraged:

In [ ]:
x, y = sm.symbols('x y')
expr = x**2 + sm.sin(y) / x**2
inputs['states'] = Values(p=expr)
display(inputs)

A Values serializes to a depth-first traversed list. This means it implements StorageOps:

In [ ]:
display(inputs.to_storage())

We can also get an equivalently flattened list of keys, with `.` separation for sub-values:

In [ ]:
display(inputs.keys_recursive())

Note that there is a `.values_recursive()`, but it's equivalent to `.to_storage()`. We can also get pairs with `.items_recursive()`:

In [ ]:
list(inputs.items_recursive())

To fully reconstruct the types in the Values from the serialized scalars, we need an index that describes which parts of the serialized list correspond to which types. The spec is `(start_index, typename, shape, item_index)`:

In [ ]:
index = inputs.index()
index

With a serialized list and an index, we can get the values back:

In [ ]:
inputs2 = Values.from_storage(inputs.values_recursive(), index)
assert inputs == inputs2
display(inputs)

The `item_index` is a recursive structure that can contain the index for a sub-values:

In [ ]:
start_index, typename, shape, item_index = inputs.index()['states']
assert item_index == inputs['states'].index()

We can also set sub-values directly with dot notation in the keys. They get split up:

In [ ]:
inputs['states.blah'] = 3
display(inputs)

The `.attr` field also allows attribute access rather than key access:

In [ ]:
assert inputs['states.p'] is inputs['states']['p'] is inputs.attr.states.p
display(inputs.attr.states.p)

Finally, SymForce adds the concept of a name scope to namespace symbols. Within a scope block, symbol names get prefixed with the scope name:

In [ ]:
with sm.scope('params'):
    s = sm.Symbol('cost')
display(s)

A common use case is to call a function that adds symbols within your own name scope to avoid name collisions. You can also chain name scopes:

In [ ]:
v = Values()
v.add(sm.Symbol('x'))
with sm.scope('foo'):
    v.add(sm.Symbol('x'))
    with sm.scope('bar'):
        v.add(sm.Symbol('x'))
display(v)
display(v.attr.foo.bar.x)

The values class also provides a `.scope()` method that not only applies the scope to symbol names but also to keys added to the Values:

In [ ]:
v = Values()
with v.scope('hello'):
    v['y'] = x**2
    v['z'] = sm.Symbol('z')
v

This flexible set of features provided by the Values class allows conveniently building up large expressions, and acts as the interface to code generation.